# Retail Store 상품 분류 LoRA 모델 서빙 및 평가

---

> 이 노트북은 SageMaker Studio Notebook PyTorch 1.13 Python 3.9 docker image 와 ml.t3.medium 에서 테스트 되었습니다. :



# 1. 기본 환경 설정

In [2]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../../common_code')

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
print(f"bucket: {sagemaker_session_bucket}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker bucket: sagemaker-us-east-1-057716757052
sagemaker session region: us-east-1
bucket: sagemaker-us-east-1-057716757052


In [8]:
# model_id = "nlpai-lab/kullm-polyglot-12.8b-v2"
# model_name = model_id.split('/')[1]
# model_name = model_name.split('.')[0]
# print("model_name: ", model_name)

# 2. 파인 튜닝 모델 배포 및 추론



## 훈련 잡 이름으로 Estimator 다시 생성

- 이전 노트북에서 훈련한 잡 이름을 가져 옵니다.
- 잡 이름을 통해서 모델 정보(모델 가중치 파일 등) 를 가져와서 Estimator 를 생성

In [9]:
%store -r retail_store_training_job_name
print("retail_store_training_job_name: \n", retail_store_training_job_name)

retail_store_training_job_name: 
 kullm-polyglot-12-ko-retail-store-data--2023-09-24-08-06-25-412


In [10]:
from sagemaker.estimator import Estimator
attached_estimator = Estimator.attach(retail_store_training_job_name)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml

2023-09-24 09:07:01 Starting - Preparing the instances for training
2023-09-24 09:07:01 Downloading - Downloading input data
2023-09-24 09:07:01 Training - Training image download completed. Training in progress.
2023-09-24 09:07:01 Uploading - Uploading generated training model
2023-09-24 09:07:01 Completed - Training job completed


## SageMaker Model 을 생성하여 모델 배포

In [11]:
from sagemaker.huggingface import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=attached_estimator.model_data,
   #model_data="s3://hf-sagemaker-inference/model.tar.gz",  # Change to your model path
   role=role, 
   transformers_version="4.26", 
   pytorch_version="1.13", 
   py_version="py39",
   model_server_workers=1
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


We can now deploy our model using the `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

## 앤드포인트 이름 생성

In [12]:
model_name = 'fine-tuned-kullm-polyglot-12-8b-v2'

In [13]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
fine_tuned_model_endpoint_name = f"{model_name}-" + time_stamp

In [14]:
%%time

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   endpoint_name= fine_tuned_model_endpoint_name,    
   initial_instance_count=1,
   instance_type= "ml.g5.4xlarge"
)

-----------!CPU times: user 144 ms, sys: 15.4 ms, total: 160 ms
Wall time: 6min 3s


### 아래는 실험을 위하여 하드 코딩 하였습니다. 
- 실제 작업시에 주석 처리하고 사용 하세요.

In [13]:
fine_tuned_model_endpoint_name = 'fine-tuned-kullm-polyglot-12-8b-v2-2023-09-24-09-18-08'

# 3. 베이스 모델 추론

## [중요] 아래 노트북을 실행한 후에 생성되는 엔드포인트 이름을 아래에 넣어 주세요.
- Kor-LLM-On-SageMaker/1-Lab01-Deploy-LLM/1.Deploy-Kor-LLM-PythonSDK.ipynb
- 이후에 아래 셀에 하드코딩된 엔드포인트 이름은 지워 주세요.

In [14]:
base_model_endpoint_name = '<Type Your Endpoint Name'
base_model_endpoint_name = 'Kullm-polyglot-12-8b-v2-2023-09-24-10-48-02'


In [15]:
def create_prompt(prompt, is_fine_tuned=False):
    '''
    구름의 모델 경우는 베이스 모델 엔드포인트 생성시에 model.py 를 자제 정의한 기준으로 엔드포인트 생성함.
    반면에 파인 튜닝시는 모델 제공자의 디폴트 모델 설정 파일로 엔드포인트를 생성하기에 "입력 프로프트" 의 정의가 약간 다름. 그래서 아래 처럼 2가지로 구분하여 프로프트 생성. 
    '''
    max_new_tokens = 64
    temperature = 0.8
    top_k = 10
    top_p = 0.5
    repetition_penalty = 1.5
    
    
    if is_fine_tuned:        
        fomatted_sample = {
          # "inputs": prompter.generate_prompt(q, c),
          "inputs": prompt,  
          "parameters": {
            "do_sample":False, 
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty,
            "presence_penalty":None,
            "eos_token_id":2,              
          }
        }
    else:
        fomatted_sample = {
          # "inputs": prompter.generate_prompt(q, c),
          "prompt": prompt,  
          "params": {
            "do_sample":False, 
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty,
            "presence_penalty":None,
            "eos_token_id":2,              
          }
        }
        
    # print("fomatted_sample: \n", fomatted_sample)
    
    return fomatted_sample

def generate_llm(prompt):
    # predict
    res = predictor.predict(prompt)

    print(res[0]["generated_text"].split("Summary:")[-1])


    

## 프롬프트 생성

In [16]:
from inference_lib import print_ww
import json

In [17]:
from inference_lib import invoke_inference_DJ,  parse_response

In [30]:
from langchain.prompts import PromptTemplate

prompt_template = """다음의 상품 설명을 분류하세요.\n\
### 상품 설명: {question}\n\n### 답변:"""

prompt_template_object = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)
# Pass in values to the input variables
q = "방을 돋보이게 해주는 세련된 게인즈버러 액센트 테이블" # 레이블: 가구|테이블
prompt_data = prompt_template_object.format(question=q)
print("prompt_data: \n", prompt_data)

prompt_data: 
 다음의 상품 설명을 분류하세요.
### 상품 설명: 방을 돋보이게 해주는 세련된 게인즈버러 액센트 테이블

### 답변:


In [31]:
fomatted_prompt = create_prompt(prompt_data, is_fine_tuned=False)
print("fomatted_prompt: : \n", fomatted_prompt)

fomatted_prompt: : 
 {'prompt': '다음의 상품 설명을 분류하세요.\n### 상품 설명: 방을 돋보이게 해주는 세련된 게인즈버러 액센트 테이블\n\n### 답변:', 'params': {'do_sample': False, 'max_new_tokens': 64, 'temperature': 0.8, 'top_k': 10, 'top_p': 0.5, 'repetition_penalty': 1.5, 'presence_penalty': None, 'eos_token_id': 2}}


In [32]:
res = invoke_inference_DJ(base_model_endpoint_name, fomatted_prompt)
res = json.loads(res)
print_ww(res[0]['generated_text'])

다음의 상품 설명을 분류하세요.
### 상품 설명: 방을 돋보이게 해주는 세련된 게인즈버러 액센트 테이블

### 답변:
방에 어울리고, 아름답지만 비싸지 않으며, 고급스럽거나 화려한 느낌이 나지않으면서도 우아하고 매력적입니다."


# 4. Fune-Tuned 모델 추론

In [35]:
fomatted_prompt = create_prompt(prompt_data, is_fine_tuned=True)
res = invoke_inference_DJ(fine_tuned_model_endpoint_name, fomatted_prompt)
res = json.loads(res)
print_ww(res[0]['generated_text'])

다음의 상품 설명을 분류하세요.
### 상품 설명: 방을 돋보이게 해주는 세련된 게인즈버러 액센트 테이블

### 답변:
가구|테이블


# 5. 베이스 모델, 튜닝 모델 동시 비교

In [51]:
def run_comppare_result(question, label, prompt_template_object, base_model_endpoint_name, fine_tuned_model_endpoint_name):
    prompt_data = prompt_template_object.format(question=question)
    
    # (1) Base Model
    fomatted_prompt = create_prompt(prompt_data, is_fine_tuned=False)
    res = invoke_inference_DJ(base_model_endpoint_name, fomatted_prompt)
    res = json.loads(res)
    print("> Base Model: \n")
    print_ww(res[0]['generated_text'])

    
    # (2) Fine-tuned Model
    fomatted_prompt = create_prompt(prompt_data, is_fine_tuned=True)
    res = invoke_inference_DJ(fine_tuned_model_endpoint_name, fomatted_prompt)
    res = json.loads(res)
    
    print("\n")
    print("> Fine-Tuned Model: \n")    
    print_ww(res[0]['generated_text'])
    


In [54]:
q = "이 페루 오렌지 드레시 핸드백은 흠잡을 데 없습니다" 
label = "액세서리|핸드백"
run_comppare_result(question = q, 
                    label = label,
                    prompt_template_object = prompt_template_object,
                    base_model_endpoint_name = base_model_endpoint_name, 
                    fine_tuned_model_endpoint_name = fine_tuned_model_endpoint_name)    

> Base Model: 

다음의 상품 설명을 분류하세요.
### 상품 설명: 이 페루 오렌지 드레시 핸드백은 흠잡을 데 없습니다

### 답변: 다음과 같으면 됩니다."흠 잡기 어려운"이라는 단어는 "완벽한", 또는 그에 상응한다고 볼 수 있으므로, 가방 자체가 완벽하고 결함이나 결점 없이 만들어졌다는 것으로
해석할 수도있고 아닐수도 있지만 후자를 선택했다고 가정해 보겠니다 :


> Fine-Tuned Model: 

다음의 상품 설명을 분류하세요.
### 상품 설명: 이 페루 오렌지 드레시 핸드백은 흠잡을 데 없습니다

### 답변:
액세서리|핸드백


In [50]:
q = "이 부드러운 시에나 소파를 즐기세요" 
label = "가구|소파"
run_comppare_result(question = q, 
                    label = label,
                    prompt_template_object = prompt_template_object,
                    base_model_endpoint_name = base_model_endpoint_name, 
                    fine_tuned_model_endpoint_name = fine_tuned_model_endpoint_name)    

> Base Model: 

다음의 상품 설명을 분류하세요.
### 상품 설명: 이 부드러운 시에나 소파를 즐기세요

### 답변:
이 제품은 가구가 아닙니다


> Fine-Tuned Model: 

다음의 상품 설명을 분류하세요.
### 상품 설명: 이 부드러운 시에나 소파를 즐기세요

### 답변:
가구|소파


In [48]:
q = "이 맛있는 채소는 모든 식료품 저장실에 꼭 있어야 합니다" 
label = "식료품|야채"
run_comppare_result(question = q, 
                    label = label,
                    prompt_template_object = prompt_template_object,
                    base_model_endpoint_name = base_model_endpoint_name, 
                    fine_tuned_model_endpoint_name = fine_tuned_model_endpoint_name)    

> Base Model: 

다음의 상품 설명을 분류하세요.
### 상품 설명: 이 맛있는 채소는 모든 식료품 저장실에 꼭 있어야 합니다

### 답변: 맛이 좋고 신선한 녹색 잎채소를 찾으신다면 브로콜리와 케일은 훌륭합니다! 두 가지 모두 비타민, 미네랄 및 기타 영양소가 풍부하여 건강에 도움되며 다양하고 풍미있게
요리할 수 있습니다."


> Fine-Tuned Model: 

다음의 상품 설명을 분류하세요.
### 상품 설명: 이 맛있는 채소는 모든 식료품 저장실에 꼭 있어야 합니다

### 답변:
식료품|야채


# 8. 엔드포인트 삭제
- SageMaker Console 에 가셔서 반드시 지위시기 바랍니다. 돈이 무지하게 발생 합니다.

![endpont.png](img/endpont.png)